# CS 109A/AC 209A/STAT 121A Data Science: Final Project
**Harvard University**<br>
**Fall 2016**<br>
**Instructors:** W. Pan, P. Protopapas, K. Rader<br>
**Members: ** Shawn Pan, Xinyuan (Amy) Wang, Ming-long Wu

## Remark ##

- Implemented sklearn model selection to rank importance of the predictors
- For OLS, the R^2 is 0.65


In [1]:
import numpy as np
import pandas as pd
import itertools as it
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
import scipy as sp
import statsmodels.api as sm
from itertools import combinations
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from collections import Counter
%matplotlib inline

In [2]:
#read the data
df = pd.read_csv('listings_clean.csv')
print(df.columns.values)
df.head()

['id' 'host_since' 'zipcode' 'latitude' 'longitude' 'property_type'
 'room_type' 'accommodates' 'bathrooms' 'bedrooms' 'beds' 'bed_type'
 'guests_included' 'minimum_nights' 'maximum_nights' 'availability_30'
 'availability_60' 'availability_90' 'availability_365' 'number_of_reviews'
 'first_review' 'last_review' 'review_scores_rating'
 'review_scores_accuracy' 'review_scores_cleanliness'
 'review_scores_checkin' 'review_scores_communication'
 'review_scores_location' 'review_scores_value' 'host_listing_count'
 'price']


,id,host_since,zipcode,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,...,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,host_listing_count,price
0,1069266,1926,10022.0,40.756852,-73.964754,0,0,2,1.0,1.0,...,2542.0,86.0,9.0,7.0,9.0,9.0,10.0,9.0,1,160
1,1846722,1625,NaN,40.830599,-73.941014,0,0,10,1.0,3.0,...,2554.0,85.0,8.0,8.0,9.0,8.0,7.0,8.0,2,105
2,2061725,1831,11221.0,40.692189,-73.924120,0,1,2,1.0,1.0,...,2554.0,98.0,10.0,10.0,10.0,10.0,9.0,10.0,4,58
3,44974,953,10011.0,40.734751,-74.002592,0,0,2,1.0,1.0,...,2494.0,96.0,10.0,9.0,10.0,10.0,10.0,9.0,1,185
4,4701675,2479,10011.0,40.745282,-73.997836,0,0,2,1.0,1.0,...,2533.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1,195


In [111]:
#transform location with KNN
#TODO this should only be done on the train set
location_model = KNeighborsRegressor(n_neighbors=50, weights="uniform")
x = df[["latitude", "longitude"]]
y = np.log(df["price"])

location_model.fit(x, y)
print location_model.score(x, y)

df.loc[:,'loc_knn'] = pd.Series(location_model.predict(x), index=df.index)

0.336228336147


In [112]:
y = df['price']

In [113]:
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin

class LongLatKNNTransformer(KNeighborsRegressor, TransformerMixin):
    def __init__(self):
        pass
    
    def fit_transform(self, X, y):
        self.fit(X, y)
        return self.predict(X)
    
    def transform(self, X):
        return self.predict(X)
    
A = LongLatKNNTransformer()

In [114]:
#Feature Extraction:
#Use the features we decided last time
features = [#'zipcode', 'latitude', 'longitude',
            'loc_knn',
            'host_since', 'minimum_nights', 'maximum_nights',
            'availability_30', 'availability_60', 'availability_90', 'availability_365',
            'first_review', 'last_review', 
            'property_type', 'room_type', 'accommodates', 
            'bathrooms', 'bedrooms', 'beds', 'bed_type', 'number_of_reviews', 'review_scores_rating',
            'review_scores_accuracy', 'review_scores_cleanliness', 
            'review_scores_checkin', 'review_scores_communication', 
            'review_scores_location', 'review_scores_value', 'host_listing_count']

df_x = df[features]
df_x.head()

,loc_knn,host_since,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,first_review,last_review,...,bed_type,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,host_listing_count
0,5.137155,1926,3,14,21,51,72,322,1944.0,2542.0,...,0,62,86.0,9.0,7.0,9.0,9.0,10.0,9.0,1
1,4.510273,1625,1,180,28,58,88,348,2196.0,2554.0,...,0,22,85.0,8.0,8.0,9.0,8.0,7.0,8.0,2
2,4.365902,1831,3,30,4,13,26,227,2226.0,2554.0,...,0,35,98.0,10.0,10.0,10.0,10.0,9.0,10.0,4
3,5.507856,953,10,30,1,1,1,274,1011.0,2494.0,...,0,26,96.0,10.0,9.0,10.0,10.0,10.0,9.0,1
4,5.280080,2479,1,1125,30,60,90,365,2533.0,2533.0,...,0,1,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1


Now we start imputing missing values. We fill in numerical values with the mean of its columns, fill in categorical values with most common value of its column.

In [115]:
#categorical and numerical column names
cate = ['property_type', 'room_type', 'bed_type'] #, 'zipcode'] 
nume = [c for c in df_x.columns.values if c not in cate]

#fill categorical with mode, numerical with mean
df_x_cate = df_x[cate].apply(lambda x:x.fillna(x.value_counts().index[0]))
df_x_nume = df_x[nume].apply(lambda x:x.fillna(x.mean()))

df_x = pd.concat([df_x_cate, df_x_nume], axis=1)
df_x.head()

,property_type,room_type,bed_type,loc_knn,host_since,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,...,beds,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,host_listing_count
0,0,0,0,5.137155,1926,3,14,21,51,72,...,1.0,62,86.0,9.0,7.0,9.0,9.0,10.0,9.0,1
1,0,0,0,4.510273,1625,1,180,28,58,88,...,3.0,22,85.0,8.0,8.0,9.0,8.0,7.0,8.0,2
2,0,1,0,4.365902,1831,3,30,4,13,26,...,2.0,35,98.0,10.0,10.0,10.0,10.0,9.0,10.0,4
3,0,0,0,5.507856,953,10,30,1,1,1,...,1.0,26,96.0,10.0,9.0,10.0,10.0,10.0,9.0,1
4,0,0,0,5.280080,2479,1,1125,30,60,90,...,2.0,1,100.0,10.0,10.0,10.0,10.0,10.0,10.0,1


Now we encode categorical variables.

In [116]:
x = pd.get_dummies(df_x, columns=cate, drop_first=False)
print x.shape
print y.shape

(27392, 37)
(27392,)


In [159]:
from sklearn.feature_selection import RFE
#RFE(estimator, n_features_to_select=None, step=1, verbose=0)
model = LinearRegression()

selector = RFE(model, n_features_to_select=1)
selector.fit(x, np.log(y))
print selector.support_
print selector.ranking_

[ True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False]
[ 1 35 22 37 33 27 28 32 34 36 12  7  3 13 31 29 20 17 30 18 25 19 24 16 15
  9  8  6  5  2 11  4 23 10 14 21 26]


In [162]:
#ranking - if you keep out all predictors except 1
print "Order of Importance"
for c in x.columns[np.argsort(selector.ranking_)]:
    print c

Order of Importance
loc_knn
room_type_0
bedrooms
room_type_2
property_type_5
property_type_4
bathrooms
property_type_3
property_type_2
bed_type_1
room_type_1
accommodates
beds
bed_type_2
property_type_1
property_type_0
review_scores_cleanliness
review_scores_communication
review_scores_value
review_scores_accuracy
bed_type_3
minimum_nights
bed_type_0
host_listing_count
review_scores_location
bed_type_4
availability_60
availability_90
review_scores_rating
review_scores_checkin
number_of_reviews
availability_365
availability_30
first_review
host_since
last_review
maximum_nights


In [163]:
model = LinearRegression()
selector = RFE(model, n_features_to_select=15)
selector.fit(x, np.log(y))

print x.columns[selector.support_]
x2 = x[x.columns[selector.support_]]

Index([u'loc_knn', u'accommodates', u'bathrooms', u'bedrooms', u'beds',
       u'property_type_1', u'property_type_2', u'property_type_3',
       u'property_type_4', u'property_type_5', u'room_type_0', u'room_type_1',
       u'room_type_2', u'bed_type_1', u'bed_type_2'],
      dtype='object')


Now we've already preprocessed data in both files. We could start build models. We split our training dataset into train data and test data.

In [142]:
x_train, x_test, y_train, y_test = train_test_split(x2, np.log(y), test_size=0.3, random_state=0)

In [143]:
print 'train data: ', x_train.shape
print 'test data: ', x_test.shape

train data:  (19174, 15)
test data:  (8218, 15)


In [144]:
#OLS
X = sm.add_constant(x_train)
model = sm.OLS(y_train,X)
results = model.fit()
print results.summary()

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.654
Model:                            OLS   Adj. R-squared:                  0.653
Method:                 Least Squares   F-statistic:                     2411.
Date:                Wed, 07 Dec 2016   Prob (F-statistic):               0.00
Time:                        19:24:40   Log-Likelihood:                -8484.5
No. Observations:               19174   AIC:                         1.700e+04
Df Residuals:                   19158   BIC:                         1.713e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
const            -5.52e+09    1.2e+10     

$R^2$ is 0.663

In [145]:
#same OLS model with sklearn (easier to score but less detailed output)
model = LinearRegression()
model.fit(x_train, y_train)
print "Train Score", model.score(x_train, y_train)
print "Test Score", model.score(x_test, y_test)

Train Score 0.653751421704
Test Score 0.666463065665


Interestingly, for this split the test set does better than the training set.

In [146]:
#try several random seeds to check for stability and variance
for r in range(10):
    x_train, x_test, y_train, y_test = train_test_split(x2, np.log(y), test_size=0.3, random_state=r)
    model = LinearRegression()
    model.fit(x_train, y_train)
    print "Train {:.3f} Test {:.3f}".format(model.score(x_train, y_train), model.score(x_test, y_test))

Train 0.654 Test 0.666
Train 0.659 Test 0.655
Train 0.660 Test 0.652
Train 0.660 Test 0.653
Train 0.662 Test 0.647
Train 0.662 Test 0.646
Train 0.654 Test 0.666
Train 0.659 Test 0.655
Train 0.657 Test 0.658
Train 0.665 Test 0.638
